In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
%sql
CREATE SCHEMA retail_cat.silver_sch

## Data Reading

In [0]:
df = spark.read.format("parquet")\
    .load("abfss://bronze@adlscardatalake.dfs.core.windows.net/customers")

In [0]:
df.display()

In [0]:
df = df.drop("_rescued_data")

In [0]:
df = df.withColumn("domains" , split(col("email"),'@')[1])
df.display()

In [0]:
df_agg = df.groupBy("domains").agg(count(col("customer_id")).alias("total_customers")).sort("total_customers",ascending = False)
display(df_agg)

In [0]:
df_gmail = df.filter(col('domains') == "gmail.com")
df_gmail.display()
time.sleep(5)

df_yahoo = df.filter(col('domains') == "yahoo.com")
df_yahoo.display()


df_hotmail = df.filter(col('domains') == "hotmail.com")
df_hotmail.display()

In [0]:
df = df.withColumn("full_name", concat(col("first_name"), lit(" "), col("last_name")))
df = df.drop("first_name", "last_name")
df.display()

In [0]:
df.write.mode("overwrite").format("delta").save("abfss://silver@adlscardatalake.dfs.core.windows.net/customers")

In [0]:
%sql
create table if not exists retail_cat.silver_sch.customer_silver
using delta
location "abfss://silver@adlscardatalake.dfs.core.windows.net/customers"

In [0]:
%sql
select * from  retail_cat.silver_sch.customer_silver